In [1]:
import polars as pl
import plotly.express as px
import plotly.graph_objects as go
pl.Config(tbl_cols=-1)
pl.Config(tbl_rows=-1)
pl.Config(tbl_width_chars=1000)
pl.Config(fmt_str_lengths=1000)

In [2]:
def load_test_cycler_data():
    """
    load the test cycler data

    "Test_1_ArbinCyclerData_Cyc_360V250V1C_45C_00014_c3.parquet"
    need to merge with the cycle protocol
    """
    df1 = pl.read_parquet(
        "/workspaces/codespaces-jupyter/data/Test_1_ArbinCyclerData_Cyc_360V250V1C_45C_00014_c3.parquet"
    )
    df2 = pl.read_parquet(
        "/workspaces/codespaces-jupyter/data/Test_2_ArbinCyclerData_Cyc_360V250V1C_45C_00014_c3.parquet"
    )
    cycle_protocol = (
        pl.read_parquet(
           "/workspaces/codespaces-jupyter/data/cyc-lfp-ummlp_45c+LFP_UMMLP_5N6P_EL09-06.parquet"
        )
        .drop("timestamp")
        .with_columns(pl.col("Step_Index").cast(pl.Int64))
    )
    df1_new = df1.join(
        cycle_protocol, left_on="Step_Index", right_on="Step_Index"
    ).with_columns(pl.lit(0.0).alias("coulombic_efficiency"))
    df1_new = df1_new.select(sorted(df1_new.columns))
    df2 = df2.select(sorted(df2.columns))

    df2 = df2.with_columns(
        [
            pl.col("Channel").cast(pl.Int64),
            pl.col("Charge_Energy").str.strip(" ").cast(pl.Float64),
            pl.col("Data_Time").cast(pl.Int64),
        ]
    )

    raw_df = (
        pl.concat([df1_new, df2]).unique(subset=["Data_Point"]).sort(by=["Data_Point"])
    ).rename(
        {
            "Test_Time": "Test_Time(s)",
            "Step_Time": "Step_Time(s)",
            "Voltage": "Voltage(V)",
            "Current": "Current(A)",
            "Charge_Energy": "Charge_Energy(Wh)",
            "Discharge_Energy": "Discharge_Energy(Wh)",
            "Discharge_Capacity": "Discharge_Capacity(Ah)",
            "Charge_Capacity": "Charge_Capacity(Ah)",
        }
    )

    return raw_df

In [3]:
raw_df = load_test_cycler_data()

/tmp/ipykernel_1867/4252058233.py:30: DeprecationWarning: `strip` is deprecated. It has been renamed to `strip_chars`.
  pl.col("Charge_Energy").str.strip(" ").cast(pl.Float64),


In [4]:
import plotly.subplots as sp
def plot_scatter_plot_df(data, x_col, y_col, y_col2, x_title, y_title, y_title2):
    return plot_scatter_plot(data[x_col], data[y_col], data[y_col2], x_title, y_title, y_title2)

def plot_scatter_plot(x_data, y_data1, y_data2, x_title, y_title, y_title2):
    fig = sp.make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(go.Scatter(
                x=x_data,
                y=y_data1,
                mode="lines+markers",
            ), secondary_y=False)
    fig.add_trace(go.Scatter(
                x=x_data,
                y=y_data2,
                mode="lines+markers",
            ), secondary_y=True)
    return (
        fig.update_xaxes(
            title_text=x_title,
            title_font={"size": 20},
            tickmode="linear",
            tickfont={"size": 10},
            #categoryorder="array",
            #categoryarray=x_data.unique(maintain_order=True),
            showgrid=True,
            showticklabels=False
        )
        .update_layout(
            legend={
                "title_text": "",
                "orientation": "h",
                "yanchor": "bottom",
                "y": 1.02,
                "xanchor": "right",
                "x": 1,
                "font": {"size": 20},
            },
        )
        .update_yaxes(title_text=y_title, secondary_y=False)
        .update_yaxes(title_text=y_title2, secondary_y=True)
    )

In [5]:
raw_df.head(5)

Arbin,Channel,Charge_Capacity(Ah),Charge_Energy(Wh),Current(A),Cycle_Index,Data_Flags,Data_Point,Data_Time,Discharge_Capacity(Ah),Discharge_Energy(Wh),Step_Index,Step_Time(s),Test_Time(s),Voltage(V),coulombic_efficiency,epoch_time_100ns,sduName,stepName,test_name,time
str,i64,f64,f64,f64,i64,f64,i64,i64,f64,f64,i64,f64,f64,f64,f64,f64,str,str,str,datetime[ns]
"""ArbinA""",32,0.0,0.0,0.0,1,2.621443e6,0,16487737547496000,0.0,0.0,1,9.6577,9.6577,3.325565,0.0,1.6488e9,"""cyc-lfp-ummlp_45c+LFP_UMMLP_5N6P_EL09-06.sdx""","""Initial Rest""","""Cyc_360V250V1C_45C_00014_c3""",2022-04-01 00:42:34.749600
"""ArbinA""",32,1.4702e-7,4.9002e-7,0.4811717,1,3.674112e6,1,16487737648222000,0.0,0.0,2,9.6632,9.6632,3.332888,0.0,1.6488e9,"""cyc-lfp-ummlp_45c+LFP_UMMLP_5N6P_EL09-06.sdx""","""iSOH_Charge CC""","""Cyc_360V250V1C_45C_00014_c3""",2022-04-01 00:42:44.822200064
"""ArbinA""",32,0.000183,0.00061,0.481558,1,2.097152e6,2,16487737661880000,0.0,0.0,2,11.029,11.029,3.33789,0.0,1.6488e9,"""cyc-lfp-ummlp_45c+LFP_UMMLP_5N6P_EL09-06.sdx""","""iSOH_Charge CC""","""Cyc_360V250V1C_45C_00014_c3""",2022-04-01 00:42:46.188
"""ArbinA""",32,0.000526,0.001755,0.4815575,1,2.097152e6,3,16487737687515000,0.0,0.0,2,13.592499,13.592499,3.342894,0.0,1.6488e9,"""cyc-lfp-ummlp_45c+LFP_UMMLP_5N6P_EL09-06.sdx""","""iSOH_Charge CC""","""Cyc_360V250V1C_45C_00014_c3""",2022-04-01 00:42:48.751499776
"""ArbinA""",32,0.000943,0.003152,0.4815542,1,2.097152e6,4,16487737718738000,0.0,0.0,2,16.714799,16.714799,3.347895,0.0,1.6488e9,"""cyc-lfp-ummlp_45c+LFP_UMMLP_5N6P_EL09-06.sdx""","""iSOH_Charge CC""","""Cyc_360V250V1C_45C_00014_c3""",2022-04-01 00:42:51.873799936


In [9]:
def plot_cycles(step_index, limit):
    for idx, c in enumerate(sorted(raw_df.filter((pl.col("Step_Index")==step_index))["Cycle_Index"].unique())):
        if idx > limit:
            break
        data = raw_df.filter((pl.col("Step_Index")==step_index) & (pl.col("Cycle_Index")==c)).sort("Data_Time")
        step_name = data["stepName"].unique()[0]
        fig = plot_scatter_plot_df(data, "Data_Point", "Current(A)", "Voltage(V)", step_name + " "+ str(c), "Current", "Voltage")
        fig.show()

In [59]:
plot_cycles(8, 5)

In [60]:
plot_cycles(9, 5)

In [5]:
step_index = 8
cc_df = (raw_df.filter((pl.col("Step_Index")==step_index))
                 .group_by("Cycle_Index")
                 .agg(pl.concat_list(pl.col("Current(A)"), 
                      pl.col("Voltage(V)").alias("voltage"))
                      .alias("features"))
).with_columns([pl.lit("Cycle Charge CC").alias("cycle_name"), pl.col("features").count().alias("count")])

In [13]:
step_index = 9
cv_df = (raw_df.filter((pl.col("Step_Index")==step_index))
                 .group_by("Cycle_Index")
                 .agg(pl.concat_list(pl.col("Current(A)"), 
                      pl.col("Voltage(V)").alias("voltage"))
                      .alias("features"))
).with_columns([pl.lit("Cycle Charge CV").alias("cycle_name"), pl.col("features").count().alias("count")])

In [6]:
cc_df.head(3)

Cycle_Index,features,cycle_name,count
i64,list[list[f64]],str,u32
551,"[[1.438492, 2.836389], [1.446143, 2.841437], … [1.446213, 3.600004]]","""Cycle Charge CC""",1574
1260,"[[1.438164, 2.847636], [1.446215, 2.85269], … [1.446193, 3.600004]]","""Cycle Charge CC""",1574
1462,"[[1.438425, 2.851487], [1.446304, 2.856534], … [1.44623, 3.600008]]","""Cycle Charge CC""",1574


In [11]:
cc_df['features'].to_list()

1574

In [6]:
import numpy as np
import tensorflow as tf
train_data = [np.array(v) for v in cc_df['features'].to_list()]

2024-03-15 20:29:36.139417: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-15 20:29:38.222655: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-15 20:29:38.222746: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-15 20:29:38.524332: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-15 20:29:39.202315: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-15 20:29:39.203539: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [7]:
# normalize your data
normalizer = tf.keras.layers.experimental.preprocessing.Normalization()
normalizer.adapt(np.concatenate(train_data))
train_data_normalized = [normalizer(array) for array in train_data]

In [19]:
np.concatenate(train_data).shape

(319555, 2)

In [8]:

train_data_padded = tf.keras.preprocessing.sequence.pad_sequences(train_data_normalized, value=-4, dtype='float32')

In [32]:
train_data_padded_input = train_data_padded[:,:,0]
train_data_padded_output = train_data_padded[:,:,1]

In [31]:
train_data_padded[:,:,0]

array([[-4.0000000e+00, -4.0000000e+00, -4.0000000e+00, ...,
         5.8298465e-03,  6.2731295e-03,  7.4014869e-03],
       [-4.0000000e+00, -4.0000000e+00, -4.0000000e+00, ...,
         6.8373084e-03,  6.3940249e-03,  4.8089516e-03],
       [-4.0000000e+00, -4.0000000e+00, -4.0000000e+00, ...,
         1.0558201e-02,  8.7582022e-03,  9.6582016e-03],
       ...,
       [-4.0000000e+00, -4.0000000e+00, -4.0000000e+00, ...,
         7.5223827e-04, -1.4776108e-04,  2.2298491e-03],
       [-4.0000000e+00, -4.0000000e+00, -4.0000000e+00, ...,
         2.9014903e-03,  5.1582051e-03,  3.4656690e-03],
       [-4.0000000e+00, -4.0000000e+00, -4.0000000e+00, ...,
         9.8865600e-03,  8.5298447e-03,  1.1014917e-02]], dtype=float32)

In [9]:
import mlflow
import mlflow.tensorflow

In [10]:
mlflow.tensorflow.autolog()
mlflow.set_tracking_uri("http://13.83.126.56:5000/")

In [11]:
experiment_name = "lstm_timeseries"
#mlflow.create_experiment(experiment_name)
mlflow.set_experiment(experiment_name)

<Experiment: artifact_location='mlflow-artifacts:/864263443647142867', creation_time=1710486127366, experiment_id='864263443647142867', last_update_time=1710486127366, lifecycle_stage='active', name='lstm_timeseries', tags={}>

In [73]:
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, RepeatVector, Masking
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

timesteps = 228
input_dim = 1
latent_dim = 50

inputs = Input(shape=(timesteps, input_dim))
masking = Masking(mask_value=-4)(inputs)
encoded = LSTM(latent_dim)(masking)

decoded = RepeatVector(timesteps)(encoded)
decoded = LSTM(input_dim, return_sequences=True)(decoded)

sequence_autoencoder = Model(masking, decoded)
encoder = Model(inputs, encoded)
# define learning rate
learning_rate = 0.0005

# define optimizer
adam = Adam(learning_rate=learning_rate)
sequence_autoencoder.compile(optimizer=adam, loss='mse')

In [13]:
len(train_data_padded)

1574

In [74]:
with mlflow.start_run() as run:
    sequence_autoencoder.fit(train_data_padded_input[0:1200,:], train_data_padded_output[0:1200,:],
                    validation_data=(train_data_padded_input[1200:1574,:],train_data_padded_output[1200:1574,:]),
                    epochs=75,
                    batch_size=64)

Epoch 1/75
19/19 [==============================] - 5s 147ms/step - loss: 2.5956 - val_loss: 2.5672
Epoch 2/75


/home/codespace/.python/current/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning:

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.



19/19 [==============================] - 2s 112ms/step - loss: 2.4847 - val_loss: 2.5066
Epoch 3/75
19/19 [==============================] - 2s 115ms/step - loss: 2.4696 - val_loss: 2.4998
Epoch 4/75
19/19 [==============================] - 2s 112ms/step - loss: 2.4642 - val_loss: 2.4933
Epoch 5/75
19/19 [==============================] - 2s 115ms/step - loss: 2.4553 - val_loss: 2.4846
Epoch 6/75
19/19 [==============================] - 2s 114ms/step - loss: 2.4489 - val_loss: 2.4812
Epoch 7/75
19/19 [==============================] - 2s 116ms/step - loss: 2.4459 - val_loss: 2.4784
Epoch 8/75
19/19 [==============================] - 2s 111ms/step - loss: 2.4447 - val_loss: 2.4777
Epoch 9/75
19/19 [==============================] - 2s 112ms/step - loss: 2.4440 - val_loss: 2.4771
Epoch 10/75
19/19 [==============================] - 2s 114ms/step - loss: 2.4435 - val_loss: 2.4767
Epoch 11/75
19/19 [==============================] - 2s 97ms/step - loss: 2.4434 - val_loss: 2.4768
Epoch 12/7

KeyboardInterrupt: 

In [62]:
reconstruction = sequence_autoencoder.predict(train_data_padded_input[0:1200,:])
reconstruction = np.resize(reconstruction, train_data_padded_input.shape)

 1/38 [..............................] - ETA: 1s

38/38 [==============================] - 1s 19ms/step


In [61]:
sequence_autoencoder.predict(train_data_padded_input[0:1,:])

1/1 [==============================] - 0s 23ms/step


array([[[-0.40303692],
        [-0.4287328 ],
        [-0.43735495],
        [-0.43749097],
        [-0.4377284 ],
        [-0.437716  ],
        [-0.43772385],
        [-0.4377229 ],
        [-0.43772322],
        [-0.43772313],
        [-0.43772316],
        [-0.43772313],
        [-0.43772316],
        [-0.43772313],
        [-0.43772316],
        [-0.43772313],
        [-0.43772316],
        [-0.43772313],
        [-0.43772316],
        [-0.43772313],
        [-0.43772316],
        [-0.43772313],
        [-0.43772316],
        [-0.43772313],
        [-0.43772316],
        [-0.43772313],
        [-0.43772316],
        [-0.43772313],
        [-0.43772316],
        [-0.43772313],
        [-0.43772316],
        [-0.43772313],
        [-0.43772316],
        [-0.43772313],
        [-0.43772316],
        [-0.43772313],
        [-0.43772316],
        [-0.43772313],
        [-0.43772316],
        [-0.43772313],
        [-0.43772316],
        [-0.43772313],
        [-0.43772316],
        [-0

In [63]:
index = 40

In [64]:
non_padded_idx = np.where(train_data_padded_input[index]!=-4)

In [70]:
len(non_padded_idx[0])

209

In [71]:

plot_scatter_plot(np.arange(0, len(non_padded_idx[0])), train_data_padded_input[index][non_padded_idx], train_data_padded_output[index][non_padded_idx], "original sequence", "current", "voltage")

In [72]:
plot_scatter_plot(np.arange(0, len(non_padded_idx[0])), train_data_padded_input[index][non_padded_idx], reconstruction[index][non_padded_idx], "reconstructed sequence", "current", "voltage")

In [30]:
train_data_normalized[0][:,0]

<tf.Tensor: shape=(198,), dtype=float32, numpy=
array([-1.0104547e+00, -2.4810361e-02,  4.5832675e-02,  1.1014887e-03,
        1.1579065e-02,  1.9692469e-02,  1.7892474e-02,  2.0364108e-02,
        2.3520814e-02,  2.2284998e-02,  2.3977529e-02,  2.3977529e-02,
        2.5777522e-02,  2.8705871e-02,  2.4984987e-02,  2.9941687e-02,
        2.5884984e-02,  2.8705871e-02,  2.8598407e-02,  2.6570057e-02,
        2.7134234e-02,  2.9605867e-02,  2.6570057e-02,  2.7241696e-02,
        2.8034231e-02,  2.9498406e-02,  3.0962579e-02,  2.8370051e-02,
        2.9605867e-02,  2.8598407e-02,  3.2762572e-02,  2.8934227e-02,
        2.8477512e-02,  2.8598407e-02,  3.0170044e-02,  3.0290939e-02,
        3.0962579e-02,  2.8477512e-02,  3.0398402e-02,  2.7577516e-02,
        3.2426752e-02,  3.0962579e-02,  3.4119286e-02,  2.8934227e-02,
        2.9041691e-02,  3.0505864e-02,  3.0626759e-02,  2.8370051e-02,
        2.6570057e-02,  2.8598407e-02,  3.1526756e-02,  3.1298399e-02,
        3.1526756e-02,  3.119

In [ ]:
# Spark imports
from pyspark.sql import SparkSession, Window, functions as func
from pyspark.sql.functions import explode, udf
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, ArrayType
from delta import *

# Other imports
import pandas as pd
import numpy as np
import sys
import json
import datetime
import time
from datetime import datetime
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np
import polars as pl
import pyarrow as pa

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
all_events_df = spark.read.parquet('dbfs:/FileStore/Prasanth/sdf_events')

In [ ]:
all_events_df.head(5)

In [ ]:
all_events_df = all_events_df.withColumn('date', func.udf(lambda f: datetime.fromtimestamp(f/1000000).strftime('%Y-%m-%d %H:%M:%S'))(func.col('timestamp')))

In [ ]:
current_df = all_events_df.filter(func.col('signal_name').isin(['I_pack_high_accuracy']))
temperature_df = all_events_df.filter(func.col('signal_name').isin(['T_module_1_thermistor_1']))

In [ ]:
datetime_format = "%Y-%m-%d %H:%M:%S"
current_df_pl = pl.from_arrow(pa.Table.from_batches(current_df._collect_as_arrow())).with_columns([pl.col('date').str.strptime(pl.Datetime, fmt=datetime_format).alias('time_current'), pl.col('signal_value').alias('current_value')]).drop('signal_value')
temperature_df_pl = pl.from_arrow(pa.Table.from_batches(temperature_df._collect_as_arrow())).with_columns([pl.col('date').str.strptime(pl.Datetime, fmt=datetime_format).alias('time_temp'), pl.col('signal_value').alias('temp_value')]).drop('signal_value')

In [ ]:
temperature_df_pl.head(5)

In [ ]:
joined_df = (temperature_df_pl.sort('time_temp')
        .join_asof(
            other=current_df_pl.sort('time_current'),
            left_on='time_temp',
            right_on='time_current',
            strategy="backward",
            by='event_id'
        ))

In [ ]:
joined_df.head(5)

In [ ]:
vectorized_df = joined_df.groupby('event_id', maintain_order=True).agg(pl.concat_list(pl.col('time_current'), pl.col('current_value'), pl.col('temp_value')).alias('features'), pl.count('time_current').alias('count'))

In [ ]:
vectorized_df

In [ ]:
def subsample(data, n):
    # Sort the data by the timestamp column
    data = sorted(data, key=lambda x: x[0])
    # Calculate the step size for subsampling
    step = len(data) // n
    # Subsample the data
    subsampled_data = data[::step]
    return subsampled_data[:n]

sampled_df = vectorized_df.filter(pl.col('count')> 100).select(pl.all(), pl.col('features').apply(lambda r: subsample(r, 100)).alias('sampled_features'))

In [ ]:
import numpy as np
train_data = np.array(sampled_df['sampled_features'].to_list())[:,:,1:3]

In [21]:
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, RepeatVector
from tensorflow.keras.models import Model

timesteps = 100
input_dim = 2
latent_dim = 50

inputs = Input(shape=(None, input_dim))
encoded = LSTM(latent_dim)(inputs)

decoded = RepeatVector(timesteps)(encoded)
decoded = LSTM(input_dim, return_sequences=True)(decoded)

sequence_autoencoder = Model(inputs, decoded)
encoder = Model(inputs, encoded)

sequence_autoencoder.compile(optimizer='adam', loss='mse')

In [ ]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)

In [22]:
norm = tf.keras.layers.experimental.preprocessing.Normalization()
norm.adapt(train_data)
normalized_data = norm(train_data)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).

In [ ]:
normalized_data

In [23]:
sequence_autoencoder.fit(train_data, train_data,
                epochs=50,
                batch_size=256,
                shuffle=True)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).

In [ ]:
reconstructions = sequence_autoencoder.predict(normalized_data)
mse = np.mean(np.square(normalized_data - reconstructions), axis=(1,2))

In [ ]:
mse

In [ ]:
event_ids = sampled_df['event_id'].to_list()

In [ ]:
for indx in range(0, len(mse)):
  fig = make_subplots(rows=2, cols=1,
                              shared_xaxes=True,
                              vertical_spacing=0.06, subplot_titles=['current', 'temperature'], x_title='', y_title='')
  fig.add_trace(go.Scatter(x=[r for r in range(0,101)], y=train_data[indx][:,0]), row=1, col=1)
  fig.update_layout(yaxis1=dict(range=[0,500]))
  fig.add_trace(go.Scatter(x=[r for r in range(0,101)], y=train_data[indx][:,1]), row=2, col=1)
  fig.update_layout(title='MSE: for {} is {}'.format(event_ids[indx], round(mse[indx],2)))
  fig.show()

In [ ]:
%sh
sudo apt-get update
sudo apt-get -y install mdbtools
sudo apt-get install -y unixodbc-dev

Hit:1 https://repos.azul.com/zulu/deb stable InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Reading package lists...


W: https://repos.azul.com/zulu/deb/dists/stable/InRelease: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the DEPRECATION section in apt-key(8) for details.


Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libmdb3 libmdbsql3
Suggested packages:
  bash-completion
The following NEW packages will be installed:
  libmdb3 libmdbsql3 mdbtools
0 upgraded, 3 newly installed, 0 to remove and 4 not upgraded.
Need to get 136 kB of archives.
After this operation, 486 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libmdb3 amd64 1.0.0+dfsg-1 [68.9 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libmdbsql3 amd64 1.0.0+dfsg-1 [18.8 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 mdbtools amd64 1.0.0+dfsg-1 [48.3 kB]


debconf: delaying package configuration, since apt-utils is not installed


Fetched 136 kB in 0s (294 kB/s)
Selecting previously unselected package libmdb3:amd64.
(Reading database ... 91138 files and directories currently installed.)
Preparing to unpack .../libmdb3_1.0.0+dfsg-1_amd64.deb ...
Unpacking libmdb3:amd64 (1.0.0+dfsg-1) ...
Selecting previously unselected package libmdbsql3:amd64.
Preparing to unpack .../libmdbsql3_1.0.0+dfsg-1_amd64.deb ...
Unpacking libmdbsql3:amd64 (1.0.0+dfsg-1) ...
Selecting previously unselected package mdbtools.
Preparing to unpack .../mdbtools_1.0.0+dfsg-1_amd64.deb ...
Unpacking mdbtools (1.0.0+dfsg-1) ...
Setting up libmdb3:amd64 (1.0.0+dfsg-1) ...
Setting up libmdbsql3:amd64 (1.0.0+dfsg-1) ...
Setting up mdbtools (1.0.0+dfsg-1) ...
Processing triggers for man-db (2.10.2-1) ...
Processing triggers for libc-bin (2.35-0ubuntu3.1) ...
Reading package lists...
Building dependency tree...
Reading state information...
unixodbc-dev is already the newest version (2.3.9-5).
0 upgraded, 0 newly installed, 0 to remove and 4 not upgra

In [ ]:
%sh
which mdb-export

/usr/bin/mdb-export
